In [1]:
import numpy as np 
import pandas as pd
from flask import jsonify

In [2]:
balls = pd.read_csv("balls.csv")
matches = pd.read_csv("matches.csv")
ipl = balls.merge(matches,left_on="match_id",right_on="id",how="inner")


In [3]:
ipl.columns

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batter', 'bowler', 'non_striker', 'batsman_runs', 'extra_runs',
       'total_runs', 'extras_type', 'is_wicket', 'player_dismissed',
       'dismissal_kind', 'fielder', 'id', 'season', 'city', 'date',
       'match_type', 'player_of_match', 'venue', 'team1', 'team2',
       'toss_winner', 'toss_decision', 'winner', 'result', 'result_margin',
       'target_runs', 'target_overs', 'super_over', 'method', 'umpire1',
       'umpire2'],
      dtype='object')

In [4]:
subset_venue_data = ipl[ipl["venue"] == "MA Chidambaram Stadium, Chepauk, Chennai"].groupby(["match_id",'inning']).agg({'total_runs':'sum','is_wicket':'sum'}).reset_index().set_index("match_id")
subset_venue_data = subset_venue_data[subset_venue_data['inning'].isin([1,2])]
subset_venue_data

,inning,total_runs,is_wicket
match_id,,,
1254058,1,159,9
1254058,2,160,8
1254060,1,187,6
1254060,2,177,5
1254062,1,152,10
1254062,2,142,7
1254063,1,149,8
1254063,2,143,9
1254066,1,150,5


In [5]:

Average_First_inning_score = str(subset_venue_data[subset_venue_data['inning']==1][["total_runs","is_wicket"]].mean().round(0).astype(int).to_dict().get('total_runs'))+'/'+str(subset_venue_data[subset_venue_data['inning']==1][["total_runs","is_wicket"]].mean().round(0).astype(int).to_dict().get('is_wicket'))
Average_second_inning_score = str(subset_venue_data[subset_venue_data['inning']==2][["total_runs","is_wicket"]].mean().round(0).astype(int).to_dict().get('total_runs'))+'/'+str(subset_venue_data[subset_venue_data['inning']==2][["total_runs","is_wicket"]].mean().round(0).astype(int).to_dict().get('is_wicket'))

Average_second_inning_score

'152/6'

In [6]:
def venuestats(venue):
    try:
        # Creating a Subset 
        subset_venue_data = ipl[ipl["venue"] == venue].groupby(["match_id",'inning']).agg({'total_runs':'sum','is_wicket':'sum'}).reset_index().set_index("match_id")

        # Excluding Super Overs 
        subset_venue_data = subset_venue_data[subset_venue_data['inning'].isin([1,2])]

        # First Inning Score 
        Average_First_inning_score = str(subset_venue_data[subset_venue_data['inning']==1][["total_runs","is_wicket"]].mean().round(0).astype(int).to_dict().get('total_runs'))+'/'+str(subset_venue_data[subset_venue_data['inning']==1][["total_runs","is_wicket"]].mean().round(0).astype(int).to_dict().get('is_wicket'))

        # Second Inning score 
        Average_second_inning_score = str(subset_venue_data[subset_venue_data['inning']==2][["total_runs","is_wicket"]].mean().round(0).astype(int).to_dict().get('total_runs'))+'/'+str(subset_venue_data[subset_venue_data['inning']==2][["total_runs","is_wicket"]].mean().round(0).astype(int).to_dict().get('is_wicket'))

        # Highest Score by Batsman
        Highest_Batsman_Score = ipl[ipl["venue"] == venue].groupby(["match_id",'batter']).agg({"batsman_runs":"sum"}).sort_values("batsman_runs",ascending=False).reset_index().drop(columns='match_id').iloc[0].to_dict()

        # Max Wickets by a Bowler
        Max_Wickets = ipl[(ipl["venue"]==venue)&(ipl['dismissal_kind'].isin(['caught', 'bowled','lbw','stumped', 'caught and bowled']))&(ipl['is_wicket']==1)].groupby("bowler").agg({"is_wicket":"sum"}).sort_values(by="is_wicket",ascending=False).reset_index().rename(columns={"is_wicket":"Wickets"}).iloc[0].to_dict()

        # Best Bowling Figure
        bowling_fig_df = ipl[(ipl["venue"]==venue)&(~ipl["dismissal_kind"].isin(['run out','retired hurt','obstructing the field','retired out']))].groupby(["match_id","bowler"]).agg({'is_wicket':"sum","total_runs":"sum"}).reset_index().sort_values(by=["is_wicket","total_runs"],ascending=[False,True]).drop(columns="match_id").rename(columns={"bowler":"Bowler","is_wicket":"Wickets","total_runs":"Runs Conceeded"})

        bowling_fig_df["Bowling Fig"] = bowling_fig_df["Wickets"].astype(str)+'/'+bowling_fig_df["Runs Conceeded"].astype(str)
        bowling_fig_df.drop(columns=["Wickets","Runs Conceeded"],inplace=True)
        bowling_fig_df.iloc[0].to_dict()

        # Best Team Score
        highest_score_team = ipl[ipl["venue"]==venue].groupby(["id","inning","batting_team"]).agg({"total_runs":"sum",'is_wicket':"sum"}).reset_index().sort_values(by=["total_runs","is_wicket"],ascending=[False,True]).drop(columns=["id","inning"])
        highest_score_team["Score"]=highest_score_team["total_runs"].astype(str)+"/"+highest_score_team["is_wicket"].astype(str)
        highest_score_team = highest_score_team.drop(columns=["total_runs","is_wicket"]).iloc[0].to_dict()

        venue_dict = {"First Inning Average Score":str(Average_First_inning_score),
                    "Second Inning Average Score":str(Average_second_inning_score),
                    "Highest Individual Score":str(Highest_Batsman_Score),
                    "Most Wickets Taken":str(Max_Wickets),
                    "Best Bowling Figure":str(bowling_fig_df.iloc[0].to_dict()),
                    "Highest Team Score":str(highest_score_team)
                    }
        

        return venue_dict
    except Exception as e:
        return f"Some Error Occurred : {e}"

    





In [7]:
ipl.columns

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batter', 'bowler', 'non_striker', 'batsman_runs', 'extra_runs',
       'total_runs', 'extras_type', 'is_wicket', 'player_dismissed',
       'dismissal_kind', 'fielder', 'id', 'season', 'city', 'date',
       'match_type', 'player_of_match', 'venue', 'team1', 'team2',
       'toss_winner', 'toss_decision', 'winner', 'result', 'result_margin',
       'target_runs', 'target_overs', 'super_over', 'method', 'umpire1',
       'umpire2'],
      dtype='object')

In [8]:
ipl[(ipl["venue"]=="M Chinnaswamy Stadium")&(ipl['dismissal_kind'].isin(['caught', 'bowled','lbw','stumped', 'caught and bowled']))].groupby(["bowler","match_id"]).agg({"is_wicket":"count","total_runs":"sum"})

is_wicket  total_runs
bowler         match_id                       
A Ashish Reddy 548356            1           0
               829719            1           0
A Choudhary    1082633           2           0
               1082636           1           0
A Kumble       336026            2           0
...                            ...         ...
Z Khan         548324            1           0
               548367            1           0
               598045            1           0
               598068            4           0
               980921            1           0

[414 rows x 2 columns]

In [9]:
bowling_fig_df = ipl[(ipl["venue"]=="M Chinnaswamy Stadium")&(~ipl["dismissal_kind"].isin(['run out','retired hurt','obstructing the field','retired out']))].groupby(["match_id","bowler"]).agg({'is_wicket':"sum","total_runs":"sum"}).reset_index().sort_values(by=["is_wicket","total_runs"],ascending=[False,True]).drop(columns="match_id").rename(columns={"bowler":"Bowler","is_wicket":"Wickets","total_runs":"Runs Conceeded"})

bowling_fig_df["Bowling Fig"] = bowling_fig_df["Wickets"].astype(str)+'/'+bowling_fig_df["Runs Conceeded"].astype(str)
bowling_fig_df.drop(columns=["Wickets","Runs Conceeded"],inplace=True)
bowling_fig_df.iloc[0].to_dict()


{'Bowler': 'S Badree', 'Bowling Fig': '4/10'}

In [10]:
# .sort_values(by="is_wicket",ascending=False).reset_index().rename(columns={"is_wicket":"Wickets"}).iloc[0].to_dict()

In [11]:
highest_score_team = ipl[ipl["venue"]=="M Chinnaswamy Stadium"].groupby(["id","inning","batting_team"]).agg({"total_runs":"sum",'is_wicket':"sum"}).reset_index().sort_values(by=["total_runs","is_wicket"],ascending=[False,True]).drop(columns=["id","inning"])
highest_score_team["Score"]=highest_score_team["total_runs"].astype(str)+"/"+highest_score_team["is_wicket"].astype(str)
highest_score_team.drop(columns=["total_runs","is_wicket"]).iloc[0].to_dict()

{'batting_team': 'Royal Challengers Bangalore', 'Score': '263/5'}

In [12]:
ipl["venue"].unique()

array(['M Chinnaswamy Stadium',
       'Punjab Cricket Association Stadium, Mohali', 'Feroz Shah Kotla',
       'Wankhede Stadium', 'Eden Gardens', 'Sawai Mansingh Stadium',
       'Rajiv Gandhi International Stadium, Uppal',
       'MA Chidambaram Stadium, Chepauk', 'Dr DY Patil Sports Academy',
       'Newlands', "St George's Park", 'Kingsmead', 'SuperSport Park',
       'Buffalo Park', 'New Wanderers Stadium', 'De Beers Diamond Oval',
       'OUTsurance Oval', 'Brabourne Stadium',
       'Sardar Patel Stadium, Motera', 'Barabati Stadium',
       'Brabourne Stadium, Mumbai',
       'Vidarbha Cricket Association Stadium, Jamtha',
       'Himachal Pradesh Cricket Association Stadium', 'Nehru Stadium',
       'Holkar Cricket Stadium',
       'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
       'Subrata Roy Sahara Stadium',
       'Maharashtra Cricket Association Stadium',
       'Shaheed Veer Narayan Singh International Stadium',
       'JSCA International Stadium Complex', 'Sh

In [16]:
def venuestats(venue):
        # Creating a Subset 
        subset_venue_data = ipl[ipl["venue"] == venue].groupby(["match_id",'inning']).agg({'total_runs':'sum','is_wicket':'sum'}).reset_index().set_index("match_id")

        # Excluding Super Overs 
        subset_venue_data = subset_venue_data[subset_venue_data['inning'].isin([1,2])]

        # First Inning Score 
        Average_First_inning_score = str(subset_venue_data[subset_venue_data['inning']==1][["total_runs","is_wicket"]].mean().round(0).astype(int).to_dict().get('total_runs'))+'/'+str(subset_venue_data[subset_venue_data['inning']==1][["total_runs","is_wicket"]].mean().round(0).astype(int).to_dict().get('is_wicket'))

        # Second Inning score 
        Average_second_inning_score = str(subset_venue_data[subset_venue_data['inning']==2][["total_runs","is_wicket"]].mean().round(0).astype(int).to_dict().get('total_runs'))+'/'+str(subset_venue_data[subset_venue_data['inning']==2][["total_runs","is_wicket"]].mean().round(0).astype(int).to_dict().get('is_wicket'))

        # Highest Score by Batsman
        Highest_Batsman_Score = ipl[ipl["venue"] == venue].groupby(["match_id",'batter']).agg({"batsman_runs":"sum"}).sort_values("batsman_runs",ascending=False).reset_index().drop(columns='match_id').iloc[0].to_dict()

        # Max Wickets by a Bowler
        Max_Wickets = ipl[(ipl["venue"]==venue)&(ipl['dismissal_kind'].isin(['caught', 'bowled','lbw','stumped', 'caught and bowled']))&(ipl['is_wicket']==1)].groupby("bowler").agg({"is_wicket":"sum"}).sort_values(by="is_wicket",ascending=False).reset_index().rename(columns={"is_wicket":"Wickets"}).iloc[0].to_dict()

        # Best Bowling Figure
        bowling_fig_df = ipl[(ipl["venue"]==venue)&(~ipl["dismissal_kind"].isin(['run out','retired hurt','obstructing the field','retired out']))].groupby(["match_id","bowler"]).agg({'is_wicket':"sum","total_runs":"sum"}).reset_index().sort_values(by=["is_wicket","total_runs"],ascending=[False,True]).drop(columns="match_id").rename(columns={"bowler":"Bowler","is_wicket":"Wickets","total_runs":"Runs Conceeded"})

        bowling_fig_df["Bowling Fig"] = bowling_fig_df["Wickets"].astype(str)+'/'+bowling_fig_df["Runs Conceeded"].astype(str)
        bowling_fig_df.drop(columns=["Wickets","Runs Conceeded"],inplace=True)
        bowling_fig_df.iloc[0].to_dict()

        # Best Team Score
        highest_score_team = ipl[ipl["venue"]==venue].groupby(["id","inning","batting_team"]).agg({"total_runs":"sum",'is_wicket':"sum"}).reset_index().sort_values(by=["total_runs","is_wicket"],ascending=[False,True]).drop(columns=["id","inning"])
        highest_score_team["Score"]=highest_score_team["total_runs"].astype(str)+"/"+highest_score_team["is_wicket"].astype(str)
        highest_score_team = highest_score_team.drop(columns=["total_runs","is_wicket"]).iloc[0].to_dict()

        venue_dict = {"First Inning Average Score":str(Average_First_inning_score),
                    "Second Inning Average Score":str(Average_second_inning_score),
                    "Highest Individual Score":str(Highest_Batsman_Score),
                    "Most Wickets Taken":str(Max_Wickets),
                    "Best Bowling Figure":str(bowling_fig_df.iloc[0].to_dict()),
                    "Highest Team Score":str(highest_score_team)
                    }
        return venue_dict
venuestats('MA Chidambaram Stadium')

{'First Inning Average Score': '151/5',
 'Second Inning Average Score': '134/6',
 'Highest Individual Score': "{'batter': 'SR Watson', 'batsman_runs': 96}",
 'Most Wickets Taken': "{'bowler': 'Imran Tahir', 'Wickets': 14}",
 'Best Bowling Figure': "{'Bowler': 'Imran Tahir', 'Bowling Fig': '4/12'}",
 'Highest Team Score': "{'batting_team': 'Chennai Super Kings', 'Score': '205/5'}"}

In [ ]:
def batsman_profile(batsman_name):
    try:
        # Number of Innings
        num_innings = ipl[ipl["batter"]==batsman_name]["id"].nunique()

        # Number of Runs
        batsman_runs = ipl[ipl["batter"]==batsman_name]['batsman_runs'].sum()

        # Balls_faced
        Balls_faced = ipl[ipl["batter"]==batsman_name].shape[0] - ipl[(ipl["batter"]==batsman_name)&(ipl["extras_type"]=='wides')].shape[0]
        # Strike Rate 
        strike_rate = ((batsman_runs/Balls_faced)*100).round(2)
        # Number of times got out 
        num_outs = ipl[ipl['player_dismissed']==batsman_name].shape[0]

        # Average
        Avg = round(batsman_runs/num_outs,2)

        # Temp Dataframe
        scores_df = ipl[ipl["batter"]==batsman_name].groupby(["id"]).agg({'batsman_runs':'sum'})

        # Highest_Score
        ipl[ipl["batter"]==batsman_name].groupby(["id"]).agg({'batsman_runs':'sum'}).sort_values(ascending=False,by='batsman_runs').iloc[0]["batsman_runs"]

        # Last 5 innings
        last_five = ','.join([str(i) for i in scores_df.tail(5).reset_index()["batsman_runs"].to_list()])

        # 50s
        fifties = scores_df[(scores_df["batsman_runs"]>50)&(scores_df["batsman_runs"]<100)].shape[0]

        # 100s
        centuries = scores_df[scores_df["batsman_runs"]>100].shape[0]
        Batsman_dict = {"Innings Played":str(num_innings),
                        "Runs": str(batsman_runs),
                        "Average":str(Avg),
                        "Strike Rate":str(strike_rate),
                        "50s":str(fifties),
                        "100s":str(centuries),
                        "Last 5 Innings":last_five}
        return Batsman_dict
    except Exception as e:
        return f"Some Error Occurred {e}" 

batsman_profile("RG Sharma")


In [15]:
def BattervBatter(bat1,bat2):
    batsman1 = batsman_profile(bat1)
    batsman2 = batsman_profile(bat2)
    compare_dict = {bat1:batsman1,
                    bat2:batsman2}
    return compare_dict


In [ ]:
def bowler_profile(bowler):
    try:
        # Number of Innings
        num_innigs = ipl[ipl["bowler"]==bowler]["id"].nunique()

        # Wickets
        wickets = ipl[(ipl["bowler"]==bowler)&(~ipl["dismissal_kind"].isin(['run out','retired hurt','obstructing the field','retired out']))]["is_wicket"].sum()

        # Strike Rate 
        balls_bowled = ipl[(ipl["bowler"]==bowler)&(~ipl["extras_type"].isin(["wides","noballs"]))].shape[0]
        bowling_strike_rate = (balls_bowled/wickets).round(2)

        # Economy
        runs_conceeded = ipl[ipl["bowler"]==bowler]['total_runs'].sum()
        Bowling_Economy = ((runs_conceeded/balls_bowled)*6).round(2)

        # Best Fig.
        fig_df = ipl[ipl["bowler"]==bowler].groupby(["id"]).agg({'total_runs':'sum','is_wicket':'sum'}).sort_values(by=['is_wicket','total_runs'],ascending=[False,True])
        best_bowling = f"{fig_df["is_wicket"].iloc[0]}/{fig_df["total_runs"].iloc[0]}"

        # Recent Performance
        recent_df = ipl[ipl["bowler"]==bowler].groupby(["id"]).agg({'total_runs':'sum','is_wicket':'sum'}).tail(5).reset_index()
        fig_string = ""
        for i in range(recent_df.shape[0]):
            fig_string = fig_string+str(recent_df.iloc[i]["is_wicket"])+'/'+str(recent_df.iloc[i]["total_runs"])+","
        

        bowler_dict = {"Innings" : str(num_innigs),
                    "Wickets" : str(wickets),
                    "Strike Rate" : str(bowling_strike_rate),
                    "Economy" : str(Bowling_Economy),
                    "Best Bowling Figure": best_bowling,
                    "Recent Performance" : fig_string}
        return bowler_dict
    except Exception as e:
        return f"Some Error Occurred {e}"
bowler_profile("V Kohli")

In [21]:
def BattervBowler(bat,bowl):
        # Extracting only relevant columns
        ipl_df = ipl[['id','inning', 'over', 'ball','batter', 'bowler','batsman_runs','is_wicket','player_dismissed','dismissal_kind', 'fielder']]

        # Adding Six and Four Count
        ipl_df["Six"] = (ipl_df["batsman_runs"]==6).astype(int)

        ipl_df["Four"] = (ipl_df["batsman_runs"]==4).astype(int)

        # Grouping according to Batsman and Bowler 
        bowlervsbatter_df = ipl_df[(~ipl_df["dismissal_kind"].isin(['run out','retired hurt','obstructing the field','retired out']))].groupby(["batter","bowler"]).agg({"batsman_runs":"sum","batter":"count","is_wicket":"sum","Six":"sum","Four":"sum","id":"nunique"}).rename(columns={"batsman_runs":"Runs","batter":"Balls Faced","is_wicket":"Out","id":"Innings"}).reset_index()
        
        #  Strike Rate 
        bowlervsbatter_df["Strike Rate"] = ((bowlervsbatter_df["Runs"]/bowlervsbatter_df["Balls Faced"])*100).round(2)

        # Final Sorting
        output = bowlervsbatter_df[(bowlervsbatter_df["batter"]==bat)&(bowlervsbatter_df["bowler"]==bowl)].reset_index().drop(columns="index").transpose().to_dict().get(0)
            
        return output

BattervBowler("V Kohli","JJ Bumrah")

C:\Users\00010\AppData\Local\Temp\ipykernel_19300\1799732263.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipl_df["Six"] = (ipl_df["batsman_runs"]==6).astype(int)
C:\Users\00010\AppData\Local\Temp\ipykernel_19300\1799732263.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipl_df["Four"] = (ipl_df["batsman_runs"]==4).astype(int)


{'batter': 'V Kohli',
 'bowler': 'JJ Bumrah',
 'Runs': 145,
 'Balls Faced': 98,
 'Out': 5,
 'Six': 5,
 'Four': 16,
 'Innings': 16,
 'Strike Rate': 147.96}

In [ ]:
def BowlervBowler(bowl1,bowl2):
    bowler1 = bowler_profile(bowl1)
    bowler2 = bowler_profile(bowl2)
    compare_dict = {bowl1:bowler1,
                    bowl2:bowler2}
    return compare_dict
BowlervBowler("JJ Bumrah","RG Sharma")

In [ ]:
def BattervBowler(bat,bowl):
    # Extracting only relevant columns
    ipl_df = ipl[['id','inning', 'over', 'ball','batter', 'bowler','batsman_runs','is_wicket','player_dismissed','dismissal_kind', 'fielder','Six','Four']]

    # Adding Six and Four Count
    ipl_df["Six"] = (ipl_df["batsman_runs"]==6).astype(int)

    ipl_df["Four"] = (ipl_df["batsman_runs"]==4).astype(int)

    # Grouping according to Batsman and Bowler 
    bowlervsbatter_df = ipl_df[(~ipl_df["dismissal_kind"].isin(['run out','retired hurt','obstructing the field','retired out']))].groupby(["batter","bowler"]).agg({"batsman_runs":"sum","batter":"count","is_wicket":"sum","Six":"sum","Four":"sum","id":"nunique"}).rename(columns={"batsman_runs":"Runs","batter":"Balls Faced","is_wicket":"Out","id":"Innings"}).reset_index()
     
    #  Strike Rate 
    bowlervsbatter_df["Strike Rate"] = ((bowlervsbatter_df["Runs"]/bowlervsbatter_df["Balls Faced"])*100).round(2)

    # Final Sorting
    output = bowlervsbatter_df[(bowlervsbatter_df["batter"]==bat)&(bowlervsbatter_df["bowler"]==bowl)].reset_index().drop(columns="index").transpose().to_dict().get(0)
        
    return output
BattervBowler("MS Dhoni","JJ Bumrah")





In [ ]:
ipl.columns

ipl['dismissal_kind'].unique()


In [ ]:
ipl[(ipl['is_wicket']==1)&(ipl['dismissal_kind']=='stumped')]["fielder"].value_counts().reset_index().rename(columns={"fielder":"Player","count":"Stumpings"}).iloc[0].to_dict()

In [195]:
duck_df = ipl.groupby(["batter","id"]).agg({"batsman_runs":"sum"}).rename(columns = {"batsman_runs":"Runs"}).reset_index().drop(columns="id")


In [ ]:
duck_df[(duck_df["Runs"]>=90)&(duck_df["Runs"]<100)].value_counts().reset_index().drop(columns="Runs").rename(columns={"batter":"Batsman","count":"Times"})[["Batsman","Times"]].head()

In [ ]:
most_scored = duck_df[(duck_df["Runs"]>=90)&(duck_df["Runs"]<100)].value_counts().reset_index().drop(columns="Runs").rename(columns={"batter":"Batsman","count":"Times"})[["Batsman","Times"]].head()
most_scored_dict = {i:j for i,j in zip(most_scored["Batsman"],most_scored["Times"])}
most_scored_dict

In [ ]:
ipl.columns
# 'batter
ipl[(ipl["player_dismissed"]==ipl["batter"])&(ipl["dismissal_kind"]=="bowled")]["batter"].value_counts().reset_index().rename(columns={"batter":"Batsman","count":"Times"}).iloc[0].to_dict()

